# This is to demo GenAI prompt in FSI

## Financial Robot Advisor

When a finance event happens (such as financial report release, federal reserve rate adjustment), wealth management relationship manager (RM) needs to give financial advice to their clients, including high net wealth clients, how these events would affect their open portfolio and what action they should take.

For example, the RM's client has following portfolio:

| Equity | Amount | Average Price | Market Value |
| --- | --- | --- | --- |
| AAPL.US | 10000 | US$200.00 | US$2,000,000 |
| TSLA.US | 5000 | US$200.00 | US$1,000,000 |
| AMZN.US | 10000 | US$180.00 | US$1,800,000 |
| 0388.HK | 10000 | HK$250.00 | HK$2,500,000 |
| 0941.HK | 10000 | HK$30.00 | HK$300,000 |

| Bond | Ticker | Market Value |
| --- | --- | --- |
| Bond 30-Year 0-3/4% Treasury Inflation-Indexed Bond Due 2/15/2045 | GTII30:GOV | $4,000,000 |
| AMZN 4.250% 22Aug2057 Corp (USD) | USU02320AK24 | $1,000,000 |

When a CIO market view comes out as below
```
The Federal Reserve has likely reached the peak of its rate-hiking cycle, having raised rates by a cumulative 5.25% since the beginning of 2022 to combat inflation. With inflation steadily trending towards the 2% target and labor market conditions easing, the Fed kept rates unchanged at 5.25%-5.50% in December and hinted at potential rate cuts in the future.

The Fed's "dot plot" suggests rate cuts may occur slower than initially expected, with the median FOMC member expecting a year-end federal funds rate of 5.0% in 2024, implying one cut this year. The Fed has also lowered its inflation forecasts for 2023, 2024, and 2025, acknowledging that inflation is falling faster than expected and forecasting a soft-landing scenario.
```

Based on this market view, RM would need to interpret how it impacts the client's portfolio and how the client need to adjust her/his portfolio.

GenAI might be able to help with prompt below.

In [1]:
import json
import boto3
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
endpoint_name = 'jumpstart-dft-hf-llm-mistral-7b-ins-20240621-021805'
model_name='huggingface-llm-mistral-7b-instruct-20240621-021806'
def query_endpoint(payload):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, 
        ContentType='application/json', 
        Body=json.dumps(payload).encode('utf-8'),
        InferenceComponentName=model_name
    )
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions[0]['generated_text']
    print (
        f"Generated Text: {bold}{generated_text}{unbold}{newline}")

In [2]:
prompt = """
You are CFA analyst with global financial exposure, helping a high-net-wealth client managing portfolio as below. The client is risk-averse.

Open position is as below:
Stock:
AMZN.US, 10000 shares, US$170, US$1700000
TSLA.US, 5000 shares, US$200, US$1000000 
VZ.US, 12000 shares, US$50, US$600000 
DIS.US, 3000 shares, US$140, US$420000 
0388.HK,10000 shares, HK$250, HK$2500000
0700.HK, 1000 shares, HK$300, HK$300000

Bond:
30-Year 0-3/4% Treasury Inflation-Indexed Bond Due 2/15/2045, GTII30:GOV, US$40000000
AMZN 4.250% 22Aug2057 Corp (USD), USU02320AK24, US$10000000

Following is the most recent CIO market view.
<view>
The Federal Reserve has likely reached the peak of its rate-hiking cycle, having raised rates by a cumulative 5.25% since the beginning of 2022 to combat inflation. With inflation steadily trending towards the 2% target and labor market conditions easing, the Fed kept rates unchanged at 5.25%-5.50% in December and hinted at potential rate cuts in the future.

The Fed's "dot plot" suggests rate cuts may occur slower than initially expected, with the median FOMC member expecting a year-end federal funds rate of 5.0% in 2024, implying one cut this year. The Fed has also lowered its inflation forecasts for 2023, 2024, and 2025, acknowledging that inflation is falling faster than expected and forecasting a soft-landing scenario.
</view>

Based on the CIO market view, please give advice to the risk averse c
"""


payload = {
    "inputs": prompt,
    "parameters":{
        "max_new_tokens": 2000,
        "return_full_text": False,
        "do_sample": True,
        "top_k":10
        }
}


In [3]:
query_endpoint(payload)

Generated Text: lient on the following:

1. Maintain, sell or buy the existing position (stocks or bonds)
2. Add position or sector exposure (recommended stocks or sectors to consider)
3. Consider alternative investments.

Answer:
Based on the CIO market view and the client's risk-averse nature, here is my advice:

1. Existing position:
Given the Fed's recent dovish tone, the client may consider holding on to their current positions, as there is a potential for stocks to rebound in a less uncertain economic environment. However, it would be prudent to monitor the inflation numbers closely, as a sudden surge could prompt the Fed to reverse its course on rate cuts. Additionally, the client could consider hedging their stock positions with some protective puts to mitigate downside risk.

2. Add position or sector exposure:
In this market environment, I would recommend adding some defensive sectors to the portfolio, such as Healthcare, Utilities, and Consumer Staples, which are likely to p